# Demo of model with an example

In [1]:
import pandas as pd

In [2]:
with open ('05_how-to-do-a-standup.en.txt') as f:
    #data = f.readlines()
    new_line = f.read().replace('\n',' ')

In [3]:
new_line = [new_line]

In [67]:
df = pd.DataFrame(new_line,columns=['transcripts'])

## Modeling

In [5]:
pip install keyphrase-vectorizers


[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: /Users/zhouwei/opt/anaconda3/envs/nlp_env/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install keybert


[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: /Users/zhouwei/opt/anaconda3/envs/nlp_env/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
from keyphrase_vectorizers import KeyphraseCountVectorizer

/Users/zhouwei/opt/anaconda3/envs/nlp_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import spacy

In [10]:
from keybert import KeyBERT

In [9]:
vectorizer = KeyphraseCountVectorizer()
nlp=spacy.load('en_core_web_sm')

In [11]:
kw_model = KeyBERT()

In [12]:
res = kw_model.extract_keywords(docs=new_line,vectorizer=KeyphraseCountVectorizer())
res

[('screen recording tool', 0.4079),
 ('recording', 0.4017),
 ('stand', 0.3796),
 ('biweekly stand', 0.371),
 ('webcam', 0.3562)]

In [65]:
d = dict(res)
maximum = max(d.values())
k = [k for k,v in d.items() if v==maximum]
type(k)

list

### Conclusion: The key word extracted from the video is 'screen recording tool'.

## Create URL for Key Word

In [23]:
import wikipedia

In [17]:
# STEP1: All keywords input will get a list of options output
def get_options(keyword):
    options = list(keyword)
    
    try:
        page = wikipedia.page(keyword).url
        
    except wikipedia.exceptions.DisambiguationError as e:
        options = e.options
    except wikipedia.PageError as e:
        options = wikipedia.search(keyword)
    return options

In [20]:
# STEP2: Calculate the similarity between original keyword and list of options output
def cosine_similarity(doc1,doc2): 
    nlp1 = nlp(doc1)
    nlp2 = nlp(doc2)
    similarity = nlp1.similarity(nlp2)
    return similarity

In [21]:
# STEP3: Create a function to evaluate option in options 
def select_option(options,keyword): #options is a list

    if len(options)==1:
        option = options[0] # The only result will be used as searching query
        
    elif len(options)>1: 
        try: 
            temp=[]
            for option in options: 
                score=cosine_similarity(wikipedia.summary(option,sentences=1),keyword)
                temp.append(score)
            option = options[temp.index(max(temp))]
            
        except:
            option = None
    
    return option

In [90]:
# Step4: Find URL for query
def create_url(query):
    page = wikipedia.page(query).url
    
    return page

In [76]:
df['key_word'] = [k]*1

In [83]:
df['options'] = df['key_word'].apply(get_options)

In [84]:
df

,transcripts,key_word,options
0,I mentioned to you that we're going to do some...,[screen recording tool],[screen recording tool]


In [86]:
df['query'] = df.apply(lambda x: select_option(x.options,x.key_word),axis=1)

In [91]:
df['URL'] = df['query'].apply(create_url)

In [92]:
df

,transcripts,key_word,options,query,URL
0,I mentioned to you that we're going to do some...,[screen recording tool],[screen recording tool],screen recording tool,https://en.wikipedia.org/wiki/Screencast


### Conslution: Finally we find the URLs for the key word 'screen recording tool'.

In [94]:
df.URL

0    https://en.wikipedia.org/wiki/Screencast
Name: URL, dtype: object